In [5]:
import math
import numpy as np
import matplotlib.pyplot as plt
import simpy
from collections import deque

loc_lambda = 2
loc_mu = 3
loc_n_servers = 5
loc_ro = loc_lambda/(loc_mu*loc_n_servers)


def customer_arrivals(env, theater):
    """Create new *moviegoers* until the sim time reaches 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        #movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)




queue = deque(np.zeros(2))
for i in range(loc_n_servers):
    queue.append(i)
    print(queue.popleft())
    
print(queue)
    
    
"""
λ – the arrival rate into the system as a whole.
μ – the capacity of each of n equal servers.
ρ represents the system load. In a single server system, it will be: ρ=λ/μ
In a multi-server system (one queue with n equal servers, each with capacity μ), it will be
ρ=λ/(nμ)
"""

0.0
0.0
0
1
2
deque([3, 4])


'\nλ – the arrival rate into the system as a whole.\nμ – the capacity of each of n equal servers.\nρ represents the system load. In a single server system, it will be: ρ=λ/μ\nIn a multi-server system (one queue with n equal servers, each with capacity μ), it will be\nρ=λ/(nμ)\n'

In [91]:

import random

import simpy


#RANDOM_SEED = 42
NEW_CUSTOMERS = 15  # Total number of customers
INTERVAL_CUSTOMERS = 5.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
Banktime = 5


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=Banktime)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))


# Setup and start the simulation
print('Bank renege')
#random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 0.6304 Customer01: Here I am
 2.0818 Customer01: RENEGED after  1.451
 4.6611 Customer00: Finished
 8.4513 Customer02: Here I am
 8.4513 Customer02: Waited  0.000
11.3747 Customer02: Finished
12.1480 Customer03: Here I am
12.1480 Customer03: Waited  0.000
13.9700 Customer03: Finished
21.9845 Customer04: Here I am
21.9845 Customer04: Waited  0.000
26.8360 Customer05: Here I am
28.2516 Customer05: RENEGED after  1.416
28.7216 Customer04: Finished
41.5793 Customer06: Here I am
41.5793 Customer06: Waited  0.000
42.4336 Customer06: Finished
42.7644 Customer07: Here I am
42.7644 Customer07: Waited  0.000
42.7778 Customer07: Finished
43.7191 Customer08: Here I am
43.7191 Customer08: Waited  0.000
45.4404 Customer08: Finished
46.7128 Customer09: Here I am
46.7128 Customer09: Waited  0.000
48.0292 Customer10: Here I am
50.4040 Customer10: RENEGED after  2.375
51.0552 Customer11: Here I am
52.7827 Customer09: Finished
5